# PCMCI+ Causal Discovery - Interactive Demo

This notebook demonstrates the high-performance C implementation of PCMCI+ for causal discovery in time series data.

**Features:**
- Intel MKL-accelerated linear algebra
- OpenMP parallelization
- Spearman rank correlation (robust to outliers)
- Benjamini-Hochberg FDR correction

In [ ]:
# Setup - add current directory to path
import sys
import os
sys.path.insert(0, os.getcwd())

import numpy as np
import matplotlib.pyplot as plt

# Import PCMCI+
from pcmci import PCMCI, run_pcmci, version
from visualize import plot_graph, plot_matrix, plot_lag_functions, print_parents

print(f"PCMCI+ Library Version: {version()}")
print(f"NumPy Version: {np.__version__}")

## 1. Synthetic Data: Simple Causal Chain

Let's create a simple causal chain: **X0 → X1 → X2 → X3**

Each variable also has autoregressive dynamics (depends on its own past).

In [ ]:
def generate_causal_chain(n_vars=4, T=1000, seed=42):
    """
    Generate a causal chain: X0 -> X1 -> X2 -> X3
    Each variable also has AR(1) dynamics.
    """
    np.random.seed(seed)
    data = np.zeros((n_vars, T))
    
    auto_coef = 0.5   # Autoregressive coefficient
    cross_coef = 0.4  # Cross-variable causal effect
    noise_std = 0.5
    
    for t in range(1, T):
        for v in range(n_vars):
            # Autoregressive term
            data[v, t] = auto_coef * data[v, t-1]
            # Causal influence from previous variable
            if v > 0:
                data[v, t] += cross_coef * data[v-1, t-1]
            # Noise
            data[v, t] += noise_std * np.random.randn()
    
    return data

# Generate data
data = generate_causal_chain(n_vars=4, T=1000)
var_names = ['X0', 'X1', 'X2', 'X3']

print(f"Data shape: {data.shape} (n_vars, T)")
print(f"\nTrue causal structure:")
print("  X0(t-1) → X0(t)  [autoregressive]")
print("  X0(t-1) → X1(t)  [causal]")
print("  X1(t-1) → X1(t)  [autoregressive]")
print("  X1(t-1) → X2(t)  [causal]")
print("  X2(t-1) → X2(t)  [autoregressive]")
print("  X2(t-1) → X3(t)  [causal]")
print("  X3(t-1) → X3(t)  [autoregressive]")

In [ ]:
# Visualize the time series
fig, axes = plt.subplots(4, 1, figsize=(12, 8), sharex=True)

for i, (ax, name) in enumerate(zip(axes, var_names)):
    ax.plot(data[i, :200], linewidth=0.8)
    ax.set_ylabel(name)
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Time')
plt.suptitle('Synthetic Time Series (first 200 points)', fontsize=14)
plt.tight_layout()
plt.show()

## 2. Run PCMCI+ Causal Discovery

In [ ]:
import time

# Create PCMCI object
pcmci = PCMCI(data, tau_max=3, var_names=var_names)

# Run causal discovery
start = time.perf_counter()
result = pcmci.run(
    alpha=0.05,           # Significance level
    use_spearman=True,    # Robust to outliers
    verbosity=0           # Silent
)
elapsed = time.perf_counter() - start

print(f"Runtime: {elapsed*1000:.1f} ms")
print(f"Significant links found: {result.n_significant}")

In [ ]:
# Print summary
print(result)

In [ ]:
# Print parents for each variable
print_parents(result)

## 3. Visualize Results

In [ ]:
# Causal graph
fig, ax = plot_graph(result, figsize=(10, 6))
plt.show()

In [ ]:
# Correlation matrix (strongest link per pair)
fig, ax = plot_matrix(result, matrix_type='val', figsize=(8, 6))
plt.show()

In [ ]:
# Lag functions - how correlation varies with lag
fig, axes = plot_lag_functions(result, figsize=(14, 4))
plt.show()

## 4. Synthetic Market Data Example

Let's simulate a more realistic scenario with market-like data:
- **SPY**: Stock index
- **BTC**: Cryptocurrency (follows SPY with lag)
- **TLT**: Bonds (flight to safety when VIX rises)
- **VIX**: Volatility (inverse to SPY)

In [ ]:
def generate_market_data(T=1000, seed=42):
    """
    Simulate market relationships:
    - VIX(t-1) -> SPY(t)   [negative: high vol → lower returns]
    - SPY(t-1) -> BTC(t)   [positive: stocks lead crypto]
    - VIX(t-1) -> TLT(t)   [positive: flight to safety]
    - SPY(t-1) -> VIX(t)   [negative: drops cause vol spikes]
    """
    np.random.seed(seed)
    
    n_vars = 4
    data = np.zeros((n_vars, T))
    var_names = ['SPY', 'BTC', 'TLT', 'VIX']
    
    for t in range(1, T):
        # SPY: autoregressive + inverse VIX effect
        data[0, t] = 0.5 * data[0, t-1] - 0.3 * data[3, t-1] + 0.5 * np.random.randn()
        
        # BTC: follows SPY with lag, higher noise
        data[1, t] = 0.4 * data[1, t-1] + 0.35 * data[0, t-1] + 0.8 * np.random.randn()
        
        # TLT: flight to safety (positive VIX correlation)
        data[2, t] = 0.6 * data[2, t-1] + 0.25 * data[3, t-1] + 0.4 * np.random.randn()
        
        # VIX: mean-reverting, spikes on SPY drops
        data[3, t] = 0.7 * data[3, t-1] - 0.4 * data[0, t-1] + 0.6 * np.random.randn()
    
    return data, var_names

market_data, market_names = generate_market_data(T=1000)

print("True market relationships:")
print("  VIX(t-1) → SPY(t)  [negative]")
print("  SPY(t-1) → BTC(t)  [positive]")
print("  VIX(t-1) → TLT(t)  [positive]")
print("  SPY(t-1) → VIX(t)  [negative]")

In [ ]:
# Run PCMCI+ on market data
market_result = run_pcmci(
    market_data, 
    tau_max=3, 
    alpha=0.05, 
    var_names=market_names
)

print(market_result)

In [ ]:
# Visualize market causal graph
fig, ax = plot_graph(market_result, figsize=(10, 6), 
                     title="Market Causal Relationships")
plt.show()

In [ ]:
# Show parents
print_parents(market_result)

## 5. Performance Benchmarks

In [ ]:
# Benchmark: scaling with number of variables
print("Scaling with number of variables (T=500, tau_max=3):")
print("-" * 50)

n_vars_list = [3, 5, 8, 10, 15, 20, 30]
times = []

for n_vars in n_vars_list:
    data = np.random.randn(n_vars, 500)
    
    # Warm up
    _ = run_pcmci(data, tau_max=3, alpha=0.05)
    
    # Timed run
    start = time.perf_counter()
    result = run_pcmci(data, tau_max=3, alpha=0.05)
    elapsed = time.perf_counter() - start
    
    times.append(elapsed * 1000)
    print(f"  n_vars={n_vars:2d}: {elapsed*1000:8.1f} ms  ({result.n_significant} links)")

In [ ]:
# Plot scaling
fig, ax = plt.subplots(figsize=(10, 5))

ax.semilogy(n_vars_list, times, 'bo-', linewidth=2, markersize=8)
ax.set_xlabel('Number of Variables', fontsize=12)
ax.set_ylabel('Runtime (ms)', fontsize=12)
ax.set_title('PCMCI+ Performance Scaling (T=500, τ_max=3)', fontsize=14)
ax.grid(True, alpha=0.3)

# Add reference lines
ax.axhline(10, color='green', linestyle='--', alpha=0.5, label='10 ms (real-time)')
ax.axhline(100, color='orange', linestyle='--', alpha=0.5, label='100 ms (interactive)')
ax.axhline(1000, color='red', linestyle='--', alpha=0.5, label='1 s (batch)')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Benchmark: scaling with sample size
print("\nScaling with sample size (n_vars=5, tau_max=3):")
print("-" * 50)

T_list = [200, 500, 1000, 2000, 5000]
times_T = []

for T in T_list:
    data = np.random.randn(5, T)
    
    start = time.perf_counter()
    result = run_pcmci(data, tau_max=3, alpha=0.05)
    elapsed = time.perf_counter() - start
    
    times_T.append(elapsed * 1000)
    print(f"  T={T:5d}: {elapsed*1000:8.1f} ms")

## 6. Comparing Different Alpha Levels

In [ ]:
# Generate data with known structure
data = generate_causal_chain(n_vars=4, T=1000)
var_names = ['X0', 'X1', 'X2', 'X3']

# Test different significance levels
alphas = [0.01, 0.05, 0.10, 0.20]

print("Effect of significance level (α):")
print("-" * 50)

for alpha in alphas:
    result = run_pcmci(data, tau_max=3, alpha=alpha, var_names=var_names)
    print(f"\nα = {alpha:.2f}: {result.n_significant} significant links")
    for link in result.significant_links:
        if link.pval < alpha:  # Only truly significant
            print(f"  {var_names[link.source_var]}(t-{link.tau}) → {var_names[link.target_var]}(t): "
                  f"r={link.val:.3f}, p={link.pval:.2e}")

## 7. Real Data Template

Here's how to use PCMCI+ with your own data:

In [ ]:
# Template for loading your own data

# Option 1: From CSV (each column is a variable)
# import pandas as pd
# df = pd.read_csv('your_data.csv')
# data = df.values.T  # Transpose to (n_vars, T)
# var_names = list(df.columns)

# Option 2: From numpy array
# data = np.load('your_data.npy')  # Should be shape (n_vars, T)

# Run PCMCI+
# result = run_pcmci(
#     data,
#     tau_max=5,          # Maximum lag to test
#     alpha=0.05,         # Significance level
#     var_names=var_names # Optional variable names
# )

# Visualize
# plot_graph(result)
# print_parents(result)

print("Template ready - uncomment and modify for your data!")

## 8. Export Results

In [ ]:
# Get results in different formats
data = generate_causal_chain(n_vars=4, T=1000)
result = run_pcmci(data, tau_max=3, alpha=0.05, var_names=['X0', 'X1', 'X2', 'X3'])

# As list of Link objects
print("Significant links as objects:")
for link in result.significant_links[:5]:
    print(f"  {link}")

# As numpy arrays
print(f"\nValue matrix shape: {result.val_matrix.shape}")
print(f"P-value matrix shape: {result.pval_matrix.shape}")
print(f"Adjacency matrix shape: {result.adj_matrix.shape}")

# Get parents for a specific variable
print(f"\nParents of X2: {result.get_parents(2)}")

In [ ]:
# Convert to tigramite format (for comparison)
tigramite_format = result.to_tigramite_format()
print("Tigramite-compatible format:")
for var, parents in tigramite_format.items():
    if parents:
        print(f"  Variable {var}: {parents}")

---

## Summary

This PCMCI+ implementation provides:

- **Speed**: Intel MKL + OpenMP parallelization
- **Robustness**: Spearman correlation + winsorization
- **Correctness**: FDR correction for multiple testing
- **Ease of use**: Simple Python API with visualization

Performance guidelines:
- **< 10 ms**: Up to ~10 variables (real-time)
- **< 100 ms**: Up to ~20 variables (interactive)
- **< 1 s**: Up to ~40 variables (batch)